In [2]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

## Tools

In [3]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

@tool("Web Search")
def search(query: str) -> str:
    """Searches the web for the given query and returns a summary of the results."""
    # Simulate a web search (in a real implementation, you'd call an API here)
    return f"Results for {query}"

@tool("Get Weather")
def get_weather(location: str) -> str:
    """Gets the current weather for the given location."""
    # Simulate getting weather data (in a real implementation, you'd call a weather API here)
    return f"The current weather in {location} is sunny and 75°F."

model = ChatOpenAI(model="gpt-4.1", temperature=0)

agent = create_agent(
    model=model,
    tools=[search, get_weather])

In [4]:
search.name, get_weather.name, search.description, get_weather.description, search.args, get_weather.args

('Web Search',
 'Get Weather',
 'Searches the web for the given query and returns a summary of the results.',
 'Gets the current weather for the given location.',
 {'query': {'title': 'Query', 'type': 'string'}},
 {'location': {'title': 'Location', 'type': 'string'}})

## Tool error handling

In [8]:
from textwrap import wrap
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage
from ollama import Tool

@wrap_tool_call
def handel_tool_error(request, handler):
    try:
        return handler(request)
    except Exception as e:
        return ToolMessage(
            content=f"Tool error: Please check your input and try again {str(e)}",
            tool_call_id=request.tool_call['id']
            )


agent = create_agent(
    model=model,
    tools=[search, get_weather],
    middleware=[handel_tool_error])

## Litrary agent

In [2]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.messages import SystemMessage, HumanMessage

model = ChatOpenAI(model="gpt-4.1", temperature=0)

literary_agent = create_agent(
    model=model,
    system_prompt=SystemMessage(
        content=[
            {
                "type": "text",
                "text": "You are a literary expert. tasked with analyzing literary works.",
            },
            {
                "type": "text",
                "text": "<the entire contents of 'Pride and Prejudice'>",
                "cache_control": {"type": "ephemeral"},
            },
        ]
    )
)

In [3]:
result = literary_agent.invoke(
    {"messages": [HumanMessage("Analyze the major themes in 'Pride and Prejudice'.")]}
)

In [14]:
print(result['messages'][1].content)

Certainly! *Pride and Prejudice* by Jane Austen is a rich novel with several interwoven themes. Here are the major ones:

**1. Pride and Prejudice**  
The title itself highlights the central theme: the dangers and consequences of pride and prejudice. Both Elizabeth Bennet and Mr. Darcy are guilty of these flaws. Darcy’s pride in his social status leads him to look down on the Bennet family, while Elizabeth’s prejudice against Darcy, fueled by first impressions and misleading information, blinds her to his true character. The novel traces their journeys toward self-awareness and mutual understanding.

**2. Social Class and Status**  
Austen explores the rigid class structure of early 19th-century England. The Bennet family’s lower social standing and lack of wealth threaten the daughters’ prospects. Marriages are often viewed as economic transactions, and characters like Lady Catherine de Bourgh and Mr. Collins embody the obsession with rank and propriety.

**3. Marriage and Relationshi

## Dynamic Prompt

In [15]:
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."
    if user_role == "expert":
        return base_prompt + " Provide detailed technical responses."
    elif user_role == "beginner":
        return base_prompt + " explain concepts in simple terms and avoid jargon."